In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import multivariate_normal

In [2]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import StratifiedKFold

## Read Data

### read feature

In [3]:
video_img = pd.read_csv('../data_preprocessing/files/video_img_movie_small.csv')
video_img = video_img.iloc[:, 1:]
print len(video_img)
video_img.head()

9817


,img_name,img_path,video_id,videos.type,structure.type
0,small___07_02_2017___ong-chu-bi-an-banner-107-...,data/images/videos/587d9c7e17dc133d791e7eca/sm...,587d9c7e17dc133d791e7eca,phim-bo,phim-bo
1,small___download___14-07-2015___7cxqhuug2hogbv...,data/images/videos/55a4bb5a17dc1333bea073db/sm...,55a4bb5a17dc1333bea073db,phim-bo,phim-bo
2,small___11_12_2014___twne11-12-2014_16g44-01.jpg,data/images/videos/5489670e17dc1324a4149ff7/sm...,5489670e17dc1324a4149ff7,phim-le-old,phim-le-old
3,small___08_03_2017___untitled-408-03-2017_18g0...,data/images/videos/5813565c17dc1318bc43178b/sm...,5813565c17dc1318bc43178b,phim-le,phim-le
4,small___20_04_2016___5ug5ze9720-04-2016_15g37-...,data/images/videos/57173d3117dc135b3889108b/sm...,57173d3117dc135b3889108b,phim-bo,phim-bo


In [4]:
PATH_ROOT_PROJ = '../..'

In [5]:
import sys
sys.path.append("..")

from read_path import load_img
from style_feature import Style_Feature
style_obj = Style_Feature(lab = 1, gist = 0, gist_processmodel = None)
feature = np.array([style_obj.get_feature(load_img('%s/%s' % (PATH_ROOT_PROJ, img))) for img in video_img['img_path']])

KeyboardInterrupt: 

In [ ]:
None_list = [i for i in range(len(feature)) if feature[i] is None]
print None_list

In [64]:
item = np.array(list(set(range(len(feature))) - set(None_list)))

In [129]:
np.savetxt('item_list.txt', item, fmt='%d')
item

array([   0,    1,    2, ..., 9814, 9815, 9816])

In [15]:
feature_bk = np.array(feature, copy=True)  

In [ ]:
for i in None_list:
    feature[i] = np.zeros(feature[item[0]].shape)

In [82]:
data = np.zeros((len(feature), len(feature[0])))
for i in range(len(feature)):
    for j in range(len(feature[i])):
        data[i][j] = feature[i][j]
print (data.shape)

(9817, 784)


In [108]:
np.savetxt('data.txt', data)



### prepare usr_list

In [3]:
usr_img_df = pd.read_csv('../data_preprocessing/files/usr-img_100_300.csv')
usr_img_df = usr_img_df.iloc[:, 1:]
usr_img_df.head()

,user_id,video_id,img_name,img_path
0,9796892,5898331b17dc1309f34e9c52,small___06_02_2017___tam-sinh-tam-the-thap-ly-...,data/images/videos/small___06_02_2017___tam-si...
1,322253,5898331b17dc1309f34e9c52,small___06_02_2017___tam-sinh-tam-the-thap-ly-...,data/images/videos/small___06_02_2017___tam-si...
2,4649377,5898331b17dc1309f34e9c52,small___06_02_2017___tam-sinh-tam-the-thap-ly-...,data/images/videos/small___06_02_2017___tam-si...
3,9847750,5898331b17dc1309f34e9c52,small___06_02_2017___tam-sinh-tam-the-thap-ly-...,data/images/videos/small___06_02_2017___tam-si...
4,4318646,5898331b17dc1309f34e9c52,small___06_02_2017___tam-sinh-tam-the-thap-ly-...,data/images/videos/small___06_02_2017___tam-si...


In [45]:
usr_ids = np.unique(usr_img_df['user_id'])
usr_list = []
for usr in usr_ids:
    items = usr_img_df[usr_img_df['user_id'] == usr]
    img_idx = list(set(np.array([np.where((video_img['video_id'] == items.iloc[i]['video_id']) & \
                        (video_img['img_name'] == items.iloc[i]['img_name']))[0] \
                           for i in range(len(items))]).flatten()) - set(None_list))
    usr_list.append(img_idx)

In [121]:
data_tmp = np.loadtxt('data.txt')
print np.sum([data[i] != data_tmp[i] for i in range(9817)])

0


In [112]:
import pickle

with open('usr_list.txt', 'wb') as fp:
    pickle.dump(usr_list, fp)   

In [102]:
print 'Number of user:', len(usr_list)

Number of user: 310


## GMM

In [105]:
def  gmm_usritem(item, usr_list, feature):
    '''
        itemset: list indexes of images corresponding to video_img. Discard indexes where feature is None.
        usr_list: |U| users, each user is a sets of items.
        feature: gist or lab feature coressponding image in video_img.
    '''
    for n_cluster in [12, 16, 24, 32, 36, 48, 64, 72, 80]:
        print n_cluster

        estimators = dict((cov_type, GaussianMixture(n_components=n_cluster,
                       covariance_type=cov_type, max_iter=20, random_state=0))
                      for cov_type in ['spherical', 'full'])  #'', 'diag',
        n_estimators = len(estimators)

        label_init = np.random.randint(0, n_cluster, len(item))
        label_pred = None

        for index, (name, gmm) in enumerate(estimators.items()):
            print name
            gmm.means_init = np.array([feature[item[label_init == i]].mean(axis=0) for i in range(n_cluster)])
            gmm.fit(feature[item])
            label = gmm.predict(feature[item])
            clusters = [np.where(label == i)[0] for i in range(n_cluster)]

            means = gmm.means_
            covs = gmm.covariances_

            p_us = []
            for usr in usr_list:
                p_x = [multivariate_normal.logpdf(feature[i], mean = means[label[i]], cov = covs[label[i]]) \
                       for i in usr]
                p_u = np.sum(np.array(p_x))
                p_us.append(p_u)
            print 'Avg logpdf user: ', np.array(p_us).mean()

In [106]:
gmm_usritem(item, usr_list, data)

12
spherical
Avg logpdf user:  334728.681627
full
Avg logpdf user:  560756.494043
16
spherical
Avg logpdf user:  330143.038102
full
Avg logpdf user:  571658.812033
24
spherical
Avg logpdf user:  327942.423049
full
Avg logpdf user:  559230.917406
32
spherical


KeyboardInterrupt: 

In [122]:
def  gmm_usritem(item, usr_list, feature):
    '''
        itemset: list indexes of images corresponding to video_img. Discard indexes where feature is None.
        usr_list: |U| users, each user is a sets of items.
        feature: gist or lab feature coressponding image in video_img.
    '''
    for n_cluster in [32, 36, 48, 64, 72, 80]:
        print n_cluster

        estimators = dict((cov_type, GaussianMixture(n_components=n_cluster,
                       covariance_type=cov_type, max_iter=20, random_state=0))
                      for cov_type in ['full'])  #'', 'diag',
        n_estimators = len(estimators)

        label_init = np.random.randint(0, n_cluster, len(item))
        label_pred = None

        for index, (name, gmm) in enumerate(estimators.items()):
            print name
            gmm.means_init = np.array([feature[item[label_init == i]].mean(axis=0) for i in range(n_cluster)])
            gmm.fit(feature[item])
            label = gmm.predict(feature[item])
            clusters = [np.where(label == i)[0] for i in range(n_cluster)]

            means = gmm.means_
            covs = gmm.covariances_

            p_us = []
            for usr in usr_list:
                p_x = [multivariate_normal.logpdf(feature[i], mean = means[label[i]], cov = covs[label[i]]) \
                       for i in usr]
                p_u = np.sum(np.array(p_x))
                p_us.append(p_u)
            print 'Avg logpdf user: ', np.array(p_us).mean()
gmm_usritem(item, usr_list, data)

32
full
Avg logpdf user:  541074.09214
36
full
Avg logpdf user:  545674.536314
48
full
Avg logpdf user:  523219.490253
64
full
Avg logpdf user:  515910.242452
72
full
Avg logpdf user:  504959.685178
80
full
Avg logpdf user:  475482.169995
